In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, TrainingArguments, Trainer
import spacy
import torch
from spacy.language import Language
from spacy import displacy
import time
import glob
import re
import math
import statistics
import os
import json
import calendar
import holidays
from pathlib import Path
from datetime import date
from datetime import datetime
import pandas as pd
import numpy as np
import collections
import hashlib
from dateutil.parser import parse
import shutil
import ast
from io import StringIO
import requests
import glob

In [ ]:
import spacy
from spacy.language import Language
from spacy import displacy
import time

@Language.component("newsent")
def set_custom_boundaries(doc):
    for token in doc[:-1]:
        #print(token.text, token.text in ("’s", "'s"))
        if token.text.upper() in (";", "--", "\n\n", "\n", "QUARTERLY", "STORY", "\n\n\n\n", "\n\n\n"):
            #print("Detected:", token.text)
            doc[token.i].is_sent_start = True
    return doc

#spacy.require_gpu()
nlp = spacy.load("../../Summary/NER/RelateEntity/train/model-best-local")
nlp.add_pipe('sentencizer')
nlp.add_pipe('newsent', name="newsent", last=True)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")

def query1_from_list(context):
    ans = "{\"RELATIONS\": [\"KEY:GAAP Net Income Per Share!!TYPE:OUT!!MONEY:$0.38!!LINK:KV\"]}"
    sent = ["GAAP Net Income Per Share is $0.38 in second quarter 2023 @@@"]
    
    ans1 = "{\"RELATIONS\": [\"KEY:Non-GAAP net income!!TYPE:OUT!!MONEY:$110.1 MN!!LINK:KV\"]}"
    sent1 = ["Non-GAAP net income is $110.1 million in third quarter 2022 @@@"]
    
    ans2 = "{\"RELATIONS\": [\"KEY:Non-GAAP Net Income Per Share!!TYPE:OUT!!MONEY:-$0.13!!LINK:KV\"]}"
    sent2 = ["Non-GAAP Net Income Per Share is ($0.13) in first quarter 2023 @@@"]
    
    tfewshot = f"""
    Question: What are the relations present in the following text? 
    
    Context: {" * ".join(sent)}. 
    
    Answer: {ans}.
    
    """
    tfewshot += f"""
    Question: What are the relations present in the following text? 
     
    Context: {" * ".join(sent1)}. 
    
    Answer: {ans1}.
    
    """
    tfewshot += f"""
    Question: What are the relations present in the following text?
    
    Context: {" * ".join(sent2)}. 
    
    Answer: {ans2}
    
    """
    #print(tfewshot)
    #print("\n\n")
    t5query = f"""{tfewshot}
    Question: What are the relations present in the following text? 
    
    Context:  {" * ".join(context)}.
    
    Answer:
    
    """
    print(t5query)
    inputs = tokenizer(t5query, return_tensors="pt")
    outputs = model.generate(**inputs, max_new_tokens=100)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

context = ["GAAP Gross profit for the third quarter of 2022 was $210 million"]
result = query1_from_list(context)
print(f"{result[0]}")

In [2]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", torch_dtype=torch.bfloat16)

def query1_from_list(context):
    ans = "{\"RELATIONS\": [\"KEY:GAAP Net Income Per Share!!TYPE:OUT!!MONEY:$0.38!!LINK:KV\"]}"
    sent = ["GAAP Net Income Per Share is $0.38 in second quarter 2023 @@@"]
    
    ans1 = "{\"RELATIONS\": [\"KEY:Non-GAAP net income!!TYPE:OUT!!MONEY:$110.1 MN!!LINK:KV\"]}"
    sent1 = ["Non-GAAP net income is $110.1 million in third quarter 2022 @@@"]
    
    ans2 = "{\"RELATIONS\": [\"KEY:Non-GAAP Net Income Per Share!!TYPE:OUT!!MONEY:-$0.13!!LINK:KV\"]}"
    sent2 = ["Non-GAAP Net Income Per Share is ($0.13) in first quarter 2023 @@@"]
    
    tfewshot = f"""
    Article: {" * ".join(sent)}
    
    Question: What are the relations present in the text? Display it in json format. {ans}
    
    """
    tfewshot += f"""
    Article: {" * ".join(sent1)}. 
    
    Question: What are the relations present in the text? Display it in json format. {ans1}
    
    """
    tfewshot += f"""
    Article: {" * ".join(sent2)}.
    
    Question: What are the relations present in the text? Display it in json format. {ans2}
    
    """
    #print(tfewshot)
    #print("\n\n")
    t5query = f"""{tfewshot}
    Article: {" * ".join(context)}.
    
    Question: What are the relations present in the text? Display it in json format."""
    print(t5query)
    inputs = tokenizer(t5query, return_tensors="pt")
    outputs = model.generate(**inputs, max_new_tokens=100)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

context = ["GAAP Gross profit for the third quarter of 2022 was $210 million"]
result = query1_from_list(context)
print(f"{result[0]}")


    Article: GAAP Net Income Per Share is $0.38 in second quarter 2023 @@@
    
    Question: What are the relations present in the text? Display it in json format. {"RELATIONS": ["KEY:GAAP Net Income Per Share!!TYPE:OUT!!MONEY:$0.38!!LINK:KV"]}
    
    
    Article: Non-GAAP net income is $110.1 million in third quarter 2022 @@@. 
    
    Question: What are the relations present in the text? Display it in json format. {"RELATIONS": ["KEY:Non-GAAP net income!!TYPE:OUT!!MONEY:$110.1 MN!!LINK:KV"]}
    
    
    Article: Non-GAAP Net Income Per Share is ($0.13) in first quarter 2023 @@@.
    
    Question: What are the relations present in the text? Display it in json format. {"RELATIONS": ["KEY:Non-GAAP Net Income Per Share!!TYPE:OUT!!MONEY:-$0.13!!LINK:KV"]}
    
    
    Article: GAAP Gross profit for the third quarter of 2022 was $210 million.
    
    Question: What are the relations present in the text? Display it in json format.
"RELATIONS": ["KEY:GAAP Gross Profit"]


In [2]:
devDataFile = "../../Summary/DATA/FLAN/Dev/dev.tsv"
trainDataFile = "../../Summary/DATA/FLAN/Train/train.tsv"
testDataFile = "../../Summary/DATA/FLAN/Test/test.tsv"

trainDir = "../../Summary/DATA/FLAN/Train"
devDir = "../../Summary/DATA/FLAN/Dev"
testDir = "../../Summary/DATA/FLAN/Test"

def writeTrainingData(writeFile, writeDir):
    files = glob.glob(writeDir+"/*_ER.tsv")
    print(files)
    frames = list()

    if(len(files) > 0):
        for file in files:
            print(file)
            #df = pd.read_csv(file, sep="\t", encoding = "utf-8").astype(str)
            df = pd.read_csv(file, sep="\t", encoding = "ISO-8859-1").astype(str)
            df = df.dropna()
            df = df[df['Sentence1'].notna()]
            #print(df)
            frames.append(df)
    result = pd.concat(frames)
    print(result)
    result.to_csv(writeFile, sep='\t', index=False, header=True)

In [3]:
writeTrainingData(trainDataFile, trainDir)
writeTrainingData(devDataFile, devDir)
writeTrainingData(testDataFile, testDir)

['../../Summary/DATA/FLAN/Train\\APPN_2023-02-16_EP_YH_ER.tsv', '../../Summary/DATA/FLAN/Train\\APPN_2023-05-09_EP_YH_ER.tsv', '../../Summary/DATA/FLAN/Train\\APPN_2023-08-03_EP_YH_ER.tsv', '../../Summary/DATA/FLAN/Train\\BILL_2022-08-18_EP_YH_ER.tsv', '../../Summary/DATA/FLAN/Train\\BILL_2023-02-02_EP_YH_ER.tsv', '../../Summary/DATA/FLAN/Train\\BILL_2023-05-04_EP_YH_ER.tsv', '../../Summary/DATA/FLAN/Train\\BILL_2023-08-17_EP_YH_ER.tsv', '../../Summary/DATA/FLAN/Train\\CFLT_2022-08-03_EP_YH_ER.tsv', '../../Summary/DATA/FLAN/Train\\CFLT_2022-11-02_EP_YH_ER.tsv', '../../Summary/DATA/FLAN/Train\\CFLT_2023-05-03_EP_YH_ER.tsv', '../../Summary/DATA/FLAN/Train\\CFLT_2023-08-02_EP_YH_ER.tsv', '../../Summary/DATA/FLAN/Train\\CRWD_2022-08-30_EP_YH_ER.tsv', '../../Summary/DATA/FLAN/Train\\CRWD_2022-11-29_EP_YH_ER.tsv', '../../Summary/DATA/FLAN/Train\\CRWD_2023-03-07_EP_YH_ER.tsv', '../../Summary/DATA/FLAN/Train\\CRWD_2023-08-30_EP_YH_ER.tsv', '../../Summary/DATA/FLAN/Train\\DDOG_2022-08-04_EP_YH_

In [6]:
train_df = pd.read_csv(trainDataFile, sep="\t", encoding = "utf-8").astype(str)
eval_df = pd.read_csv(devDataFile, sep="\t", encoding = "utf-8").astype(str)
test_df = pd.read_csv(testDataFile, sep="\t", encoding = "utf-8").astype(str)

train_df = train_df.rename(
    columns={"Sentence1": "input_text", "Sentence2": "target_text"}
)
eval_df = eval_df.rename(
    columns={"Sentence1": "input_text", "Sentence2": "target_text"}
)
test_df = test_df.rename(
    columns={"Sentence1": "input_text", "Sentence2": "target_text"}
)

train_df = train_df[["input_text", "target_text"]]
eval_df = eval_df[["input_text", "target_text"]]
test_df = test_df[["input_text", "target_text"]]

#train_df["prefix"] = "paraphrase"
#train_df = train_df[["prefix", "input_text", "target_text"]]
#train_df = train_df[["input_text", "target_text"]]

#eval_df["prefix"] = "paraphrase"
#eval_df = eval_df[["prefix", "input_text", "target_text"]]
#eval_df = eval_df[["input_text", "target_text"]]

train_df = train_df.dropna()
train_df = train_df[train_df['input_text'].notna()]

eval_df = eval_df.dropna()
eval_df = eval_df[eval_df['input_text'].notna()]

test_df = test_df.dropna()
test_df = test_df[test_df['input_text'].notna()]

#train_df["input_text"] = train_df["input_text"].apply(clean_unnecessary_spaces)
#train_df["target_text"] = train_df["target_text"].apply(clean_unnecessary_spaces)
print("TRAIN DATA ..............")
print(train_df)

#eval_df["input_text"] = eval_df["input_text"].apply(clean_unnecessary_spaces)
#eval_df["target_text"] = eval_df["target_text"].apply(clean_unnecessary_spaces)
print("EVAL DATA ..............")
print(eval_df)

print("TEST DATA ..............")
print(test_df)

TRAIN DATA ..............
                                             input_text  \
0     GAAP Net Loss Per Share is $(0.47) in fourth q...   
1     Cash And Cash Equivalents is $148132 T in four...   
2     GAAP Gross Profit is $90555 T in fourth quarte...   
3     GAAP Gross Margin is 71.99% in fourth quarter ...   
4                                                 PG***   
...                                                 ...   
2023  PG*** Total revenue of $1,283.0 to $1,287.0 mi...   
2024  PG*** Non-GAAP income from operations is expec...   
2025  PG*** Non-GAAP net income per share is $.37, u...   
2026  PG*** These statements are forward-looking and...   
2027  Refer to the Forward-Looking Statements safe h...   

                                            target_text  
0     {"RELATIONS": ["KEY:GAAP Net Income Per Share!...  
1     {"RELATIONS": ["KEY:Cash And Cash Equivalents!...  
2     {"RELATIONS": ["KEY:GAAP GROSS PROFIT!!TYPE:OU...  
3     {"RELATIONS": ["KEY:GAAP GR

In [3]:
model_name='google/flan-t5-small'

#original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
#tokenizer = AutoTokenizer.from_pretrained(model_name)
#model_name = "facebook/bart-base"

#original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
#original_model.to('cuda')

In [5]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 76961152
all model parameters: 76961152
percentage of trainable model parameters: 100.00%


In [25]:
inputText = (train_df["input_text"][0])
outputText = (train_df["target_text"][0])

ans = "{\"RELATIONS\": [\"KEY:GAAP Net Income Per Share!!TYPE:OUT!!MONEY:$0.38!!LINK:KV\"]}"
sent = ["GAAP Net Income Per Share is $0.38 in second quarter 2023 @@@"]

ans1 = "{\"RELATIONS\": [\"KEY:Non-GAAP net income!!TYPE:OUT!!MONEY:$110.1 MN!!LINK:KV\"]}"
sent1 = ["Non-GAAP net income is $110.1 million in third quarter 2022 @@@"]

ans2 = "{\"RELATIONS\": [\"KEY:Non-GAAP Net Income Per Share!!TYPE:OUT!!MONEY:-$0.13!!LINK:KV\"]}"
sent2 = ["Non-GAAP Net Income Per Share is ($0.13) in first quarter 2023 @@@"]

tfewshot = f"""
Article: {" * ".join(sent)}

Question: What are the relations present in the text? Display it in json format. {ans}

"""
tfewshot += f"""
Article: {" * ".join(sent1)}. 

Question: What are the relations present in the text? Display it in json format. {ans1}

"""
tfewshot += f"""
Article: {" * ".join(sent2)}.

Question: What are the relations present in the text? Display it in json format. {ans2}

"""
#print(tfewshot)
#print("\n\n")
t5query = f"""{tfewshot}
Article: {inputText}.

Question: What are the relations present in the text? Display it in json format."""
#print(t5query)
inputs = tokenizer(t5query, return_tensors="pt")
outputs = original_model.generate(**inputs, max_new_tokens=100)
output = tokenizer.batch_decode(outputs, skip_special_tokens=True)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{t5query}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{outputText}\n')
print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Article: GAAP Net Income Per Share is $0.38 in second quarter 2023 @@@

Question: What are the relations present in the text? Display it in json format. {"RELATIONS": ["KEY:GAAP Net Income Per Share!!TYPE:OUT!!MONEY:$0.38!!LINK:KV"]}


Article: Non-GAAP net income is $110.1 million in third quarter 2022 @@@. 

Question: What are the relations present in the text? Display it in json format. {"RELATIONS": ["KEY:Non-GAAP net income!!TYPE:OUT!!MONEY:$110.1 MN!!LINK:KV"]}


Article: Non-GAAP Net Income Per Share is ($0.13) in first quarter 2023 @@@.

Question: What are the relations present in the text? Display it in json format. {"RELATIONS": ["KEY:Non-GAAP Net Income Per Share!!TYPE:OUT!!MONEY:-$0.13!!LINK:KV"]}


Article: GAAP Net Loss Per Share is $(0.47) in fourth quarter 2022 @@@.

Question: What are the relations present in the text? Display it in json format.
----------

In [7]:
import datasets
from datasets import Dataset, DatasetDict

train = Dataset.from_pandas(train_df)
valid = Dataset.from_pandas(eval_df)
test = Dataset.from_pandas(test_df)

ds = DatasetDict()

ds['train'] = train
ds['validation'] = valid
ds['test'] = test

print(ds)

DatasetDict({
    train: Dataset({
        features: ['input_text', 'target_text', '__index_level_0__'],
        num_rows: 2028
    })
    validation: Dataset({
        features: ['input_text', 'target_text', '__index_level_0__'],
        num_rows: 495
    })
    test: Dataset({
        features: ['input_text', 'target_text', '__index_level_0__'],
        num_rows: 82
    })
})


In [8]:
def tokenize_function(example):
    #print(example)
    start_prompt = 'Article: '
    end_prompt = '\n\nQuestion: What are the relations present in the text? Display it in json format.'
    prompt = [start_prompt + sentence + end_prompt for sentence in example["input_text"]]
    #print(prompt)
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["target_text"], padding="max_length", truncation=True, return_tensors="pt").input_ids
    #print(example)
    return example

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = ds.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['input_text', 'target_text', '__index_level_0__',])

Map:   0%|          | 0/2028 [00:00<?, ? examples/s]

Map:   0%|          | 0/495 [00:00<?, ? examples/s]

Map:   0%|          | 0/82 [00:00<?, ? examples/s]

In [ ]:
#tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)
#tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)

In [9]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

Shapes of the datasets:
Training: (2028, 2)
Validation: (495, 2)
Test: (82, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 2028
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 495
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 82
    })
})


In [10]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
#data_collator = DataCollatorForSeq2Seq(tokenizer, model=original_model)

In [11]:
output_dir = "./feroutputs"
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_steps=1,
    predict_with_generate=True,
    overwrite_output_dir=True,
    #fp16=True,
    #max_steps=2,
    save_steps=1,
    report_to="none",
    save_total_limit=1
)

trainer = Seq2SeqTrainer(
    model=original_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer
)

In [12]:
trainer.train()

c:\users\ankan\appdata\local\programs\python\python37\lib\site-packages\transformers\optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2028
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 2028


Epoch,Training Loss,Validation Loss
1,0.026400,0.047772


Saving model checkpoint to ./feroutputs\checkpoint-1
Configuration saved in ./feroutputs\checkpoint-1\config.json
Model weights saved in ./feroutputs\checkpoint-1\pytorch_model.bin
tokenizer config file saved in ./feroutputs\checkpoint-1\tokenizer_config.json
Special tokens file saved in ./feroutputs\checkpoint-1\special_tokens_map.json
Copy vocab file to ./feroutputs\checkpoint-1\spiece.model
Saving model checkpoint to ./feroutputs\checkpoint-2
Configuration saved in ./feroutputs\checkpoint-2\config.json
Model weights saved in ./feroutputs\checkpoint-2\pytorch_model.bin
tokenizer config file saved in ./feroutputs\checkpoint-2\tokenizer_config.json
Special tokens file saved in ./feroutputs\checkpoint-2\special_tokens_map.json
Copy vocab file to ./feroutputs\checkpoint-2\spiece.model
Deleting older checkpoint [feroutputs\checkpoint-1] due to args.save_total_limit
Saving model checkpoint to ./feroutputs\checkpoint-3
Configuration saved in ./feroutputs\checkpoint-3\config.json
Model weigh

Model weights saved in ./feroutputs\checkpoint-18\pytorch_model.bin
tokenizer config file saved in ./feroutputs\checkpoint-18\tokenizer_config.json
Special tokens file saved in ./feroutputs\checkpoint-18\special_tokens_map.json
Copy vocab file to ./feroutputs\checkpoint-18\spiece.model
Deleting older checkpoint [feroutputs\checkpoint-17] due to args.save_total_limit
Saving model checkpoint to ./feroutputs\checkpoint-19
Configuration saved in ./feroutputs\checkpoint-19\config.json
Model weights saved in ./feroutputs\checkpoint-19\pytorch_model.bin
tokenizer config file saved in ./feroutputs\checkpoint-19\tokenizer_config.json
Special tokens file saved in ./feroutputs\checkpoint-19\special_tokens_map.json
Copy vocab file to ./feroutputs\checkpoint-19\spiece.model
Deleting older checkpoint [feroutputs\checkpoint-18] due to args.save_total_limit
Saving model checkpoint to ./feroutputs\checkpoint-20
Configuration saved in ./feroutputs\checkpoint-20\config.json
Model weights saved in ./ferou

Model weights saved in ./feroutputs\checkpoint-35\pytorch_model.bin
tokenizer config file saved in ./feroutputs\checkpoint-35\tokenizer_config.json
Special tokens file saved in ./feroutputs\checkpoint-35\special_tokens_map.json
Copy vocab file to ./feroutputs\checkpoint-35\spiece.model
Deleting older checkpoint [feroutputs\checkpoint-34] due to args.save_total_limit
Saving model checkpoint to ./feroutputs\checkpoint-36
Configuration saved in ./feroutputs\checkpoint-36\config.json
Model weights saved in ./feroutputs\checkpoint-36\pytorch_model.bin
tokenizer config file saved in ./feroutputs\checkpoint-36\tokenizer_config.json
Special tokens file saved in ./feroutputs\checkpoint-36\special_tokens_map.json
Copy vocab file to ./feroutputs\checkpoint-36\spiece.model
Deleting older checkpoint [feroutputs\checkpoint-35] due to args.save_total_limit
Saving model checkpoint to ./feroutputs\checkpoint-37
Configuration saved in ./feroutputs\checkpoint-37\config.json
Model weights saved in ./ferou

...

Model weights saved in ./feroutputs\checkpoint-1979\pytorch_model.bin
tokenizer config file saved in ./feroutputs\checkpoint-1979\tokenizer_config.json
Special tokens file saved in ./feroutputs\checkpoint-1979\special_tokens_map.json
Copy vocab file to ./feroutputs\checkpoint-1979\spiece.model
Deleting older checkpoint [feroutputs\checkpoint-1978] due to args.save_total_limit
Saving model checkpoint to ./feroutputs\checkpoint-1980
Configuration saved in ./feroutputs\checkpoint-1980\config.json
Model weights saved in ./feroutputs\checkpoint-1980\pytorch_model.bin
tokenizer config file saved in ./feroutputs\checkpoint-1980\tokenizer_config.json
Special tokens file saved in ./feroutputs\checkpoint-1980\special_tokens_map.json
Copy vocab file to ./feroutputs\checkpoint-1980\spiece.model
Deleting older checkpoint [feroutputs\checkpoint-1979] due to args.save_total_limit
Saving model checkpoint to ./feroutputs\checkpoint-1981
Configuration saved in ./feroutputs\checkpoint-1981\config.json
Mo

Copy vocab file to ./feroutputs\checkpoint-1995\spiece.model
Deleting older checkpoint [feroutputs\checkpoint-1994] due to args.save_total_limit
Saving model checkpoint to ./feroutputs\checkpoint-1996
Configuration saved in ./feroutputs\checkpoint-1996\config.json
Model weights saved in ./feroutputs\checkpoint-1996\pytorch_model.bin
tokenizer config file saved in ./feroutputs\checkpoint-1996\tokenizer_config.json
Special tokens file saved in ./feroutputs\checkpoint-1996\special_tokens_map.json
Copy vocab file to ./feroutputs\checkpoint-1996\spiece.model
Deleting older checkpoint [feroutputs\checkpoint-1995] due to args.save_total_limit
Saving model checkpoint to ./feroutputs\checkpoint-1997
Configuration saved in ./feroutputs\checkpoint-1997\config.json
Model weights saved in ./feroutputs\checkpoint-1997\pytorch_model.bin
tokenizer config file saved in ./feroutputs\checkpoint-1997\tokenizer_config.json
Special tokens file saved in ./feroutputs\checkpoint-1997\special_tokens_map.json
Co

Model weights saved in ./feroutputs\checkpoint-2012\pytorch_model.bin
tokenizer config file saved in ./feroutputs\checkpoint-2012\tokenizer_config.json
Special tokens file saved in ./feroutputs\checkpoint-2012\special_tokens_map.json
Copy vocab file to ./feroutputs\checkpoint-2012\spiece.model
Deleting older checkpoint [feroutputs\checkpoint-2011] due to args.save_total_limit
Saving model checkpoint to ./feroutputs\checkpoint-2013
Configuration saved in ./feroutputs\checkpoint-2013\config.json
Model weights saved in ./feroutputs\checkpoint-2013\pytorch_model.bin
tokenizer config file saved in ./feroutputs\checkpoint-2013\tokenizer_config.json
Special tokens file saved in ./feroutputs\checkpoint-2013\special_tokens_map.json
Copy vocab file to ./feroutputs\checkpoint-2013\spiece.model
Deleting older checkpoint [feroutputs\checkpoint-2012] due to args.save_total_limit
Saving model checkpoint to ./feroutputs\checkpoint-2014
Configuration saved in ./feroutputs\checkpoint-2014\config.json
Mo

Copy vocab file to ./feroutputs\checkpoint-2028\spiece.model
Deleting older checkpoint [feroutputs\checkpoint-2027] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 495
  Batch size = 1


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=2028, training_loss=2.1289265410528246, metrics={'train_runtime': 7341.1418, 'train_samples_per_second': 0.276, 'train_steps_per_second': 0.276, 'total_flos': 376985943539712.0, 'train_loss': 2.1289265410528246, 'epoch': 1.0})

In [8]:
instruct_model = AutoModelForSeq2SeqLM.from_pretrained("./feroutputs/checkpoint-2028")#, torch_dtype=torch.bfloat16)
#tokenizer = AutoTokenizer.from_pretrained("./feroutputs/checkpoint-2028")

In [14]:
index = 11
dialogue = ds['test'][index]['input_text']
human_baseline_summary = ds['test'][index]['target_text']
print(dialogue)
#prompt = dialogue

ans = "{\"RELATIONS\": [\"KEY:GAAP Net Income Per Share!!TYPE:OUT!!MONEY:$0.38!!LINK:KV\"]}"
sent = ["GAAP Net Income Per Share is $0.38 in second quarter 2023 @@@"]

ans1 = "{\"RELATIONS\": [\"KEY:Non-GAAP net income!!TYPE:OUT!!MONEY:$110.1 MN!!LINK:KV\"]}"
sent1 = ["Non-GAAP net income is $110.1 million in third quarter 2022 @@@"]

ans2 = "{\"RELATIONS\": [\"KEY:Non-GAAP Net Income Per Share!!TYPE:OUT!!MONEY:-$0.13!!LINK:KV\"]}"
sent2 = ["Non-GAAP Net Income Per Share is ($0.13) in first quarter 2023 @@@"]

tfewshot = f"""
Article: {" * ".join(sent)}

Question: What are the relations present in the text? Display it in json format. {ans}

"""
tfewshot += f"""
Article: {" * ".join(sent1)}. 

Question: What are the relations present in the text? Display it in json format. {ans1}

"""
tfewshot += f"""
Article: {" * ".join(sent2)}.

Question: What are the relations present in the text? Display it in json format. {ans2}

"""

prompt = f"""{tfewshot}
Article: {dialogue}.

Question: What are the relations present in the text? Display it in json format."""

inputs = tokenizer(prompt, return_tensors="pt")
outputs = instruct_model.generate(**inputs, max_new_tokens=100)
instruct_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
inputs = tokenizer(prompt, return_tensors="pt")
outputs = original_model.generate(**inputs, max_new_tokens=100)
original_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)

#print(prompt)
dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(prompt)
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{human_baseline_summary}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_output}')
print(dash_line)
print(f'INSTRUCT MODEL:\n{instruct_output}')

SCHQ*** Fourth Quarter 2022 Financial Results.
---------------------------------------------------------------------------------------------------

Article: GAAP Net Income Per Share is $0.38 in second quarter 2023 @@@

Question: What are the relations present in the text? Display it in json format. {"RELATIONS": ["KEY:GAAP Net Income Per Share!!TYPE:OUT!!MONEY:$0.38!!LINK:KV"]}


Article: Non-GAAP net income is $110.1 million in third quarter 2022 @@@. 

Question: What are the relations present in the text? Display it in json format. {"RELATIONS": ["KEY:Non-GAAP net income!!TYPE:OUT!!MONEY:$110.1 MN!!LINK:KV"]}


Article: Non-GAAP Net Income Per Share is ($0.13) in first quarter 2023 @@@.

Question: What are the relations present in the text? Display it in json format. {"RELATIONS": ["KEY:Non-GAAP Net Income Per Share!!TYPE:OUT!!MONEY:-$0.13!!LINK:KV"]}


Article: SCHQ*** Fourth Quarter 2022 Financial Results..

Question: What are the relations present in the text? Display it in json 